In [98]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

color_est = pd.read_csv("../data/color2emotion.csv",index_col = False)
emotion_est = pd.read_csv("../result/emtion_estimation.csv",index_col = 0)
action_est = pd.read_csv("../result/action_estimation.csv")

In [99]:
color_est

,Unnamed: 0,R,G,B,분노,두려움,행복,슬픔,열정,우울
0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,1,0,0,5,0.00,0.01,0.00,0.08,0.00,0.07
2,2,0,0,10,0.00,0.02,0.00,0.16,0.00,0.14
3,3,0,0,15,0.00,0.03,0.00,0.24,0.00,0.20
4,4,0,0,20,0.00,0.04,0.00,0.32,0.00,0.27
...,...,...,...,...,...,...,...,...,...,...
140603,2699,255,255,235,1.67,3.66,8.79,2.55,0.55,2.54
140604,2700,255,255,240,1.66,3.67,8.79,2.63,0.54,2.61
140605,2701,255,255,245,1.65,3.68,8.78,2.71,0.54,2.68
140606,2702,255,255,250,1.63,3.69,8.78,2.79,0.53,2.74


In [100]:
emotion_est.set_index(keys = ['감정'],inplace=True)
emotion_est = emotion_est.T
emotion_est


감정,행복,열정,두려움,분노,우울,슬픔
에너지,0.541582,0.533860,0.497210,0.519709,0.493126,0.492844
회복,0.550874,0.657327,0.729863,0.623987,0.620003,0.601720
순환,0.467014,0.558474,0.778007,0.774435,1.000000,0.607885
정화,0.286435,0.402557,0.458951,0.456645,0.405543,0.356631


In [101]:
emotion_est_h = pd.read_csv("../result/emotion_estimation_hand_craft.csv",index_col = 0)
#emotion_est_h.set_index(keys = ['감정'],inplace=True)
emotion_est_h = emotion_est_h.T
emotion_est_h


감정,행복,열정,두려움,분노,우울,슬픔
에너지,0.441997,1.000000,0.251791,0.566569,0.098494,0.291794
회복,0.280129,0.456525,0.277455,0.517889,0.594044,0.513508
순환,0.329806,0.405685,0.334114,0.520081,0.380115,0.353911
정화,0.415003,1.000000,0.536851,0.871676,0.387859,0.257016


In [102]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.142931,0.170405,0.227149,0.216091
1,운동,0.146344,0.134466,0.142636,0.206883
2,산책,0.138340,0.070666,0.036914,0.184480
3,수면,0.143820,0.082785,0.128648,0.185543
4,독서,0.181206,0.096831,0.077269,0.210896


In [103]:
class recommedation():
    def __init__(self,color_df ,action_df, emotion_df):
        self.color_df = color_df
        self.action_df = action_df
        self.emothon_df = emotion_df
        
    def get_color_idx(self,R,G,B):
        R = int(R/5)
        G = int(G/5)
        B = int(B/5)
        return R*52*52 + G*52 + B
         
    def get_item_est(self,R,G,B):
        item_idx = self.get_color_idx(R,G,B)
        ems = self.color_df.columns[-6:]
        
        item_est = np.array([0,0,0,0])
        for em in ems:                  
            #print(self.emothon_df[em] * self.color_df[em][item_idx])
            item_est = item_est + (self.emothon_df[em] * self.color_df[em][item_idx])
             
        # add Normalization code
        return np.array(item_est.tolist())/20    
    
        
    def sim_item(self, R, G , B):
        n_item_est = np.array(self.get_item_est(R,G,B))
        item = []
        error = []
        
        for idx in self.action_df.index:
            action_ests = np.array(self.action_df.loc[idx])
            action_name , *action_est = action_ests
            item.append(action_name)
            error.append(sum(abs(action_est-n_item_est)))
            
        min_idx = np.argmin(error)
        print(f'sim item is {item[min_idx]}')          
    
    def max_item(self, R, G , B, n):
        n_item_est = np.array(self.get_item_est(R,G,B))
        item = []
        error = []
        
        for idx in self.action_df.index:
            action_ests = np.array(self.action_df.loc[idx])
            action_name , *action_est = action_ests
            tmp = sum(sorted(abs(action_est-n_item_est),reverse=True)[:n])
            item.append(action_name)
            error.append(tmp)
        max_idx = np.argmax(error)
        print(f'sim item is {item[max_idx]}')          

    
    def complete_item(self, R, G , B, n):
        n_item_est = np.array(self.get_item_est(R,G,B))
        item = []
        error = []
        
        for idx in self.action_df.index:
            action_ests = np.array(self.action_df.loc[idx])
            action_name , *action_est = action_ests
            
            tmp = sum(sorted(abs(action_est+n_item_est),reverse=True)[:n])
            item.append(action_name)
            error.append(tmp)
            
        max_idx = np.argmax(error)
        print(f'sim item is {item[max_idx]}')          


In [104]:
XYZ = recommedation(color_est, action_est, emotion_est)
XYZ.sim_item(255,255,255)

sim item is 명상


In [105]:
hXYZ = recommedation(color_est, action_est, emotion_est_h)
hXYZ.sim_item(255,255,255)

sim item is 명상
